<a href="https://colab.research.google.com/github/Norod/hebrew-gpt_neo/blob/main/Demo_of_GRADIO2_Hebrew_GPT_Neo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using Gradio to wrap a text to text interface around GPT-2 

Check out the library on [github](https://github.com/gradio-app/gradio-UI) and see the [getting started](https://gradio.app/getting_started.html) page for more demos.

### Installs and Imports

In [4]:
!pip install -q gradio
!pip install -q transformers
!pip install -q Tokenizers

In [5]:
import gradio as gr
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

### Loading the model and creating the generate function

Note: You can also change to `hebrew-gpt_neo-tiny`, `hebrew-gpt_neo-small` or `hebrew-gpt_neo-xl`

---



> Indented block



In [6]:
#model_name = "Norod78/hebrew-gpt_neo-tiny"
model_name = "Norod78/hebrew-gpt_neo-small"
#model_name = "Norod78/hebrew-gpt_neo-xl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

In [ ]:
seed = 1000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = 0 if torch.cuda.is_available()==False else torch.cuda.device_count()

#print(f"device: {device}, n_gpu: {n_gpu}")

np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
  torch.cuda.manual_seed_all(seed)

model.to(device)

In [11]:
def generate_text(inp):
  
  encoded_prompt = tokenizer.encode(
  inp, add_special_tokens=False, return_tensors="pt")

  encoded_prompt = encoded_prompt.to(device)
  if encoded_prompt.size()[-1] == 0:
        input_ids = None
  else:
        input_ids = encoded_prompt

  #print("input_ids = " + str(input_ids))

  beam_output = model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
  output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  return ".".join(output.split(".")[:-1]) + "."

In [12]:
generate_text('שלום, זאת הדגמה של')

'שלום, זאת הדגמה של מה שאני חושב על עצמי.\nאני לא יודע אם זה נכון, אבל אני כן יודע שזה לא נכון. אני לא יכול להגיד את זה על אף אחד אחר, אני יכול רק לומר את הדברים האלה.'

###Creating the interface and launching!

In [ ]:
output_text = gr.outputs.Textbox()
gr.Interface(generate_text,"textbox", output_text, title=model_name,
             description="Go ahead and input a sentence and see what it completes \
             it with! Takes around 20s to run.").launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://31730.gradio.app
Interface loading below...


#### The model is now live on the gradio.app link shown above. Go ahead and open that in a new tab!

Please contact us [here](mailto:team@gradio.app) if you have any questions, or [open an issue](https://github.com/gradio-app/gradio-UI/issues/new/choose) at our github repo.

